# ¿Como extraemos los datos?

No guarde exactamente la query que use para extraer los datos en https://datos.bcn.cl/sparql  
pero era algo de este estilo, 

```
PREFIX bcnnorms: <http://datos.bcn.cl/ontologies/bcn-norms#>

SELECT DISTINCT ?xml
WHERE {
  ?doc bcnnorms:type <http://datos.bcn.cl/recurso/cl/norma/tipo#ley> .
  ?doc bcnnorms:hasXmlDocument ?xml .
}
LIMIT 100000
OFFSET 0

```

Originalmente no use el atributo **DISTINCT**

**Posibles mejoras**: Usar DISTINCT para obtener solo leyes disitntas, por el momento tengo bastantes   
leyes por lo que continuare con lo que tengo.

In [13]:
import pandas as pd

df = pd.read_csv("data/all_law.csv")
df.xml = df.xml.astype(str)

def get_norm(url):
    return url.split("=")[-1::][0]

df["norm"] = df.xml.apply(get_norm)
df.norm = df.norm.astype(int)

In [10]:
df

,ley,xml,norm
0,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1040386
1,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1055863
2,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1057115
3,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1040408
4,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1057087
...,...,...,...
19480,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1204681
19481,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1209400
19482,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1212623
19483,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,http://www.leychile.cl/Consulta/obtxml?opt=7&i...,1214470


Despues de varios intentos hice la siguiente función para descargar los xml,  
nuevamente deben haber formas mejroes de hacer esto pero funciona esta  
implementación



In [ ]:
import os
import time
import requests
from tqdm import tqdm

def descargar_xmls(df, path_folder="xml", sleep_time=60*5, start_idx=0,max_downloads = 500):
    os.makedirs(path_folder, exist_ok=True)

    blocked = False
    total = len(df)

    for idx in tqdm(range(start_idx, total)):
        try:
            norm = df.norm[idx]
            url = df.xml[idx]


            base_name = f"{norm}.xml"
            path = os.path.join(path_folder, base_name)

            counter = 2
            while os.path.exists(path):  # Para los duplicados
                name_with_suffix = f"{norm}_{counter}.xml"
                path = os.path.join(path_folder, name_with_suffix)
                counter += 1

            # Descargar el XML
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            xml_content = response.text

            with open(path, "w", encoding="utf-8") as f:
                f.write(xml_content)

            # Pausar cada max_downloads descargas
            if (idx + 1) % max_downloads == 0 and idx > 0:
                print(f"\n--- Pausa de {sleep_time} segundos tras {idx + 1} descargas ---")
                time.sleep(sleep_time)

        except requests.exceptions.RequestException as req_err:
            error_msg = str(req_err).lower()
            if "403" in error_msg or "blocked" in error_msg or "cloudfront" in error_msg:
                print(f"\n¡IP bloqueada! Deteniendo en índice {idx}.\n")
                with open("missing.txt","a") as miss:
                    miss.write(f"{idx} - bloqueo ip\n")
                blocked = True
                break
            else:
                print(f"\nError de request en índice {idx}: {req_err}. Se continúa con el siguiente.\n")
                with open("missing.txt","a") as miss:
                    miss.write(f"{idx} - Error de request\n")

        except Exception as e:
            print(f"\nError en índice {idx}: {e}. Se continúa con el siguiente.")
            with open("missing.txt","a") as miss:
                miss.write(f"{idx} - Error\n")

    if not blocked:
        print("\nProceso completado correctamente.")
    else:
        print(f"\nProceso detenido. Puedes retomarlo desde el índice {idx}.")
        with open("missing.txt","a") as miss:
            miss.write(f"{idx} - Proceso detenido\n")

In [11]:
path = "law_chile"
descargar_xmls(df,path)

  3%|▎         | 499/19485 [07:21<3:59:15,  1.32it/s]


--- Pausa de 300 segundos tras 500 descargas ---


  3%|▎         | 609/19485 [13:54<3:05:10,  1.70it/s]  


Error de request en índice 608: 502 Server Error: Bad Gateway for url: http://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=27665. Se continúa con el siguiente.


  5%|▌         | 999/19485 [19:39<4:06:04,  1.25it/s]

...


Error de request en índice 14712: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=164622 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187FCE0>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14713: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=164624 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x0000020053C8C160>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14714: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=164691 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187FF00>: Failed to resolve 'www.le

 76%|███████▋  | 14871/19485 [5:30:44<02:08, 35.89it/s]


Error de request en índice 14793: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=180774 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x0000020053C8D6A0>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14794: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=180775 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187C160>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14795: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=180776 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187CC00>: Failed to resolve 'www.le

 77%|███████▋  | 14920/19485 [5:30:44<1:41:11,  1.33s/it]



Error de request en índice 14893: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=186181 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187D260>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14894: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=186182 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187D6A0>: Failed to resolve 'www.leychile.cl' ([Errno 11001] getaddrinfo failed)")). Se continúa con el siguiente.

Error de request en índice 14895: HTTPConnectionPool(host='www.leychile.cl', port=80): Max retries exceeded with url: /Consulta/obtxml?opt=7&idNorma=186183 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002005187DAE0>: Failed to resolve 'www.le

In [12]:
path = "law_chile"
descargar_xmls(df,path,start_idx=14500 )

 10%|█         | 499/4985 [04:44<1:15:34,  1.01s/it]


--- Pausa de 300 segundos tras 15000 descargas ---


 20%|██        | 999/4985 [17:37<55:21,  1.20it/s]    


--- Pausa de 300 segundos tras 15500 descargas ---


 30%|███       | 1499/4985 [30:31<56:07,  1.04it/s]    

...


--- Pausa de 300 segundos tras 19000 descargas ---


 98%|█████████▊| 4908/4985 [1:55:34<00:50,  1.53it/s]  


Error de request en índice 19407: 502 Server Error: Bad Gateway for url: http://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=1086063. Se continúa con el siguiente.


100%|██████████| 4985/4985 [1:56:38<00:00,  1.40s/it]


Proceso completado correctamente.


In [20]:
missing=[608 ,  
        5779 , 
        10803, 
        13026, 
        13271, 
        18602, 
        19407 ]

descargar_xmls(df.query("index == @missing").reset_index())

 57%|█████▋    | 4/7 [00:02<00:02,  1.32it/s]


Error de request en índice 3: 500 Server Error: Internal Server Error for url: http://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=1117040. Se continúa con el siguiente.



 71%|███████▏  | 5/7 [00:03<00:01,  1.29it/s]


Error de request en índice 4: 500 Server Error: Internal Server Error for url: http://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=1070960. Se continúa con el siguiente.



100%|██████████| 7/7 [00:06<00:00,  1.14it/s]


Proceso completado correctamente.


In [ ]:
# No se encontro el xml de estas
# https://www.bcn.cl/leychile/navegar/imprimir?idNorma=1070960&idVersion=1956-09-07
# https://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=1070960


# https://www.bcn.cl/leychile/navegar/imprimir?idNorma=1117040&idVersion=1820-11-25
# https://www.leychile.cl/Consulta/obtxml?opt=7&idNorma=1117040

# Movemos los duplicados a otra carpeta

In [11]:
# Hay duplicados, por esta iteracion primera quiero continuar, por lo que eliminare o movere los duplicados

import os
import shutil
os.makedirs("copy_laws", exist_ok=True)

with os.scandir("law_chile") as directory:
    for entry in directory:
        name = entry.name
        if "_" in name:
            print(entry.name)
            shutil.move(entry.path, f"copy_laws/{name}")

        

1003600_2.xml
1004121_2.xml
1005168_2.xml
1005202_2.xml
1005225_2.xml
1005226_2.xml
1005251_2.xml
1005251_3.xml
1005318_2.xml
1005389_2.xml
1005390_2.xml
1005392_2.xml
1005871_2.xml
1005871_3.xml
1006315_2.xml
1007406_2.xml
1007837_2.xml
1007981_2.xml
1008412_2.xml
1008692_2.xml
1008867_2.xml
1009367_2.xml
1009477_2.xml
1009682_2.xml
1009826_2.xml
1010482_2.xml
1010668_2.xml
1010903_2.xml
1010960_2.xml
1010960_3.xml
1011986_2.xml
1012014_2.xml
1012765_2.xml
1013004_2.xml
1013716_2.xml
1014377_2.xml
1015552_2.xml
1015782_2.xml
1016177_2.xml
1016562_2.xml
1016846_2.xml
1017644_2.xml
1018376_2.xml
1019770_2.xml
1020621_2.xml
1020622_2.xml
1021286_2.xml
1021341_2.xml
1021341_3.xml
1021742_2.xml
1022345_2.xml
1022346_2.xml
1022392_2.xml
1022940_2.xml
1022962_2.xml
1022962_3.xml
1023193_2.xml
1023277_2.xml
1023965_2.xml
1024220_2.xml
1024821_2.xml
1025141_2.xml
1025210_2.xml
1026780_2.xml
1028013_2.xml
1028064_2.xml
1028414_2.xml
1028506_2.xml
1028995_2.xml
1031486_2.xml
1031639_2.xml
103215